# Antimicrobial Resistance Prevalence Analysis

**Research Question 1:** What is the overall resistance rate for commonly tested antibiotics? Which antibiotics show the highest resistance rates? What is the prevalence of multi-drug resistant organisms?

This analysis quantifies resistance prevalence to guide empiric therapy and identify critical resistance trends.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
%matplotlib inline
sns.set_style('whitegrid')
sns.set_palette('husl')

## 1. Load Cleaned Data

In [ ]:
df = pd.read_csv('../data/processed/amr_data_2025_cleaned.csv')

# Identify antibiotic columns
antibiotic_cols = [col for col in df.columns if ' - ' in col or col.startswith('NET_') or col.startswith('MET_')]

# Categorize results
def categorize_result(result):
    if pd.isna(result):
        return np.nan
    result_str = str(result).upper()
    if 'R' in result_str:
        return 'Resistant'
    elif 'S' in result_str:
        return 'Sensitive'
    elif 'I' in result_str:
        return 'Intermediate'
    return np.nan

for col in antibiotic_cols:
    df[col + '_Cat'] = df[col].apply(categorize_result)

print(f"Dataset loaded: {len(df)} isolates")
print(f"Antibiotics analyzed: {len(antibiotic_cols)}")

## 2. Resistance Rates by Antibiotic

In [ ]:
# Calculate resistance rates
resistance_data = []

for col in antibiotic_cols:
    cat_col = col + '_Cat'
    
    total = df[cat_col].notna().sum()
    if total < 30:  # Minimum sample size for reliable estimates
        continue
    
    resistant = (df[cat_col] == 'Resistant').sum()
    sensitive = (df[cat_col] == 'Sensitive').sum()
    intermediate = (df[cat_col] == 'Intermediate').sum()
    
    resistance_rate = (resistant / total) * 100
    sensitivity_rate = (sensitive / total) * 100
    
    # Extract antibiotic name
    ab_name = col.split(' - ')[1] if ' - ' in col else col.replace('_', ' ')
    
    # Calculate 95% CI for resistance rate (Wilson score interval)
    from statsmodels.stats.proportion import proportion_confint
    ci_low, ci_high = proportion_confint(resistant, total, alpha=0.05, method='wilson')
    
    resistance_data.append({
        'Antibiotic': ab_name,
        'Total_Tests': total,
        'Resistant': resistant,
        'Sensitive': sensitive,
        'Intermediate': intermediate,
        'Resistance_Rate': resistance_rate,
        'Sensitivity_Rate': sensitivity_rate,
        'CI_Low': ci_low * 100,
        'CI_High': ci_high * 100
    })

resistance_df = pd.DataFrame(resistance_data).sort_values('Resistance_Rate', ascending=False)

print(f"\nAnalyzed {len(resistance_df)} antibiotics with ≥30 tests")
print("\nTop 10 Most Resistant Antibiotics:")
print(resistance_df[['Antibiotic', 'Total_Tests', 'Resistance_Rate', 'CI_Low', 'CI_High']].head(10).to_string(index=False))

In [ ]:
# Visualize top 20 resistance rates
top_20 = resistance_df.head(20)

fig, ax = plt.subplots(figsize=(12, 10))

# Color code by resistance level
colors = ['#d62728' if x >= 50 else '#ff7f0e' if x >= 20 else '#2ca02c' 
          for x in top_20['Resistance_Rate']]

bars = ax.barh(range(len(top_20)), top_20['Resistance_Rate'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['Antibiotic'])
ax.set_xlabel('Resistance Rate (%)', fontsize=12)
ax.set_title('Top 20 Antibiotics by Resistance Rate', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (rate, n) in enumerate(zip(top_20['Resistance_Rate'], top_20['Total_Tests'])):
    ax.text(rate + 2, i, f'{rate:.1f}% (n={n})', va='center', fontsize=9)

# Add reference lines
ax.axvline(20, color='orange', linestyle='--', alpha=0.5, label='20% threshold')
ax.axvline(50, color='red', linestyle='--', alpha=0.5, label='50% threshold')
ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('../reports/figures/resistance_rates_top20.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nClinical Interpretation:")
print(f"- High resistance (≥50%): {(resistance_df['Resistance_Rate'] >= 50).sum()} antibiotics")
print(f"- Moderate resistance (20-49%): {((resistance_df['Resistance_Rate'] >= 20) & (resistance_df['Resistance_Rate'] < 50)).sum()} antibiotics")
print(f"- Low resistance (<20%): {(resistance_df['Resistance_Rate'] < 20).sum()} antibiotics")

## 3. Antibiotics with Highest Sensitivity (Best Treatment Options)

In [ ]:
# Identify antibiotics with highest sensitivity
best_abs = resistance_df.sort_values('Sensitivity_Rate', ascending=False).head(15)

print("Top 15 Antibiotics by Sensitivity Rate:")
print(best_abs[['Antibiotic', 'Total_Tests', 'Sensitivity_Rate', 'Resistance_Rate']].to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(best_abs)), best_abs['Sensitivity_Rate'], color='#2ca02c', alpha=0.7)
ax.set_yticks(range(len(best_abs)))
ax.set_yticklabels(best_abs['Antibiotic'])
ax.set_xlabel('Sensitivity Rate (%)', fontsize=12)
ax.set_title('Antibiotics with Highest Sensitivity Rates', fontsize=14, fontweight='bold')
ax.invert_yaxis()

for i, (rate, n) in enumerate(zip(best_abs['Sensitivity_Rate'], best_abs['Total_Tests'])):
    ax.text(rate - 5, i, f'{rate:.1f}% (n={n})', va='center', ha='right', fontsize=9, color='white', fontweight='bold')

ax.axvline(80, color='darkgreen', linestyle='--', alpha=0.5, label='80% threshold (first-line)')
ax.legend()

plt.tight_layout()
plt.savefig('../reports/figures/highest_sensitivity.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Multi-Drug Resistance (MDR) Analysis

In [ ]:
# Define MDR: resistant to ≥3 antibiotic classes
# Classify antibiotics by class
antibiotic_classes = {
    'Beta-lactams': ['Ampicillin', 'Amoxicillin', 'Amoxicillin/Clavulanic acid', 'Piperacillin', 'Piperacillin/Tazobactam'],
    'Cephalosporins': ['Ceftazidime', 'Cefixime', 'Cefotaxime', 'Ceftriaxone', 'Cefepime', 'Cefpodoxime', 'Cefoxitin'],
    'Fluoroquinolones': ['Ciprofloxacin', 'Levofloxacin', 'Norfloxacin', 'Ofloxacin'],
    'Aminoglycosides': ['Gentamicin', 'Amikacin', 'Tobramycin', 'Kanamycin'],
    'Carbapenems': ['Imipenem', 'Meropenem'],
    'Macrolides': ['Azithromycin', 'Erythromycin'],
    'Tetracyclines': ['Tetracycline', 'Doxycycline'],
    'Glycopeptides': ['Vancomycin', 'Teicoplanin'],
    'Sulfonamides': ['Trimethoprim/Sulfamethoxazole', 'Trimethoprim'],
    'Others': ['Nitrofurantoin', 'Fosfomycin', 'Chloramphenicol', 'Colistin']
}

# Count resistant antibiotics per class for each isolate
mdr_data = []

for idx, row in df.iterrows():
    resistant_classes = set()
    total_resistant = 0
    
    for class_name, abs_list in antibiotic_classes.items():
        class_resistant = False
        for ab in abs_list:
            # Find matching column
            matching_cols = [col for col in antibiotic_cols if ab in col]
            if matching_cols:
                col = matching_cols[0]
                if row[col + '_Cat'] == 'Resistant':
                    class_resistant = True
                    total_resistant += 1
        
        if class_resistant:
            resistant_classes.add(class_name)
    
    mdr_status = 'MDR' if len(resistant_classes) >= 3 else 'Non-MDR'
    
    mdr_data.append({
        'Organism': row['Organism Identified'],
        'Resistant_Classes': len(resistant_classes),
        'Total_Resistant_Antibiotics': total_resistant,
        'MDR_Status': mdr_status
    })

mdr_df = pd.DataFrame(mdr_data)

# MDR prevalence
mdr_count = (mdr_df['MDR_Status'] == 'MDR').sum()
mdr_prevalence = (mdr_count / len(mdr_df)) * 100

print(f"\n=== Multi-Drug Resistance Analysis ===")
print(f"Total isolates analyzed: {len(mdr_df)}")
print(f"MDR isolates (resistant to ≥3 classes): {mdr_count}")
print(f"MDR prevalence: {mdr_prevalence:.1f}%")
print(f"\nMDR Status Distribution:")
print(mdr_df['MDR_Status'].value_counts())

In [ ]:
# Visualize MDR distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MDR vs Non-MDR pie chart
mdr_counts = mdr_df['MDR_Status'].value_counts()
colors_mdr = ['#d62728', '#2ca02c']
axes[0].pie(mdr_counts.values, labels=mdr_counts.index, autopct='%1.1f%%', 
            colors=colors_mdr, startangle=90)
axes[0].set_title('MDR Prevalence', fontsize=12, fontweight='bold')

# Distribution of resistant classes
class_dist = mdr_df['Resistant_Classes'].value_counts().sort_index()
axes[1].bar(class_dist.index, class_dist.values, alpha=0.7, color='steelblue')
axes[1].set_xlabel('Number of Resistant Antibiotic Classes')
axes[1].set_ylabel('Number of Isolates')
axes[1].set_title('Distribution of Resistant Classes per Isolate', fontsize=12, fontweight='bold')
axes[1].axvline(3, color='red', linestyle='--', label='MDR threshold')
axes[1].legend()

for i, v in enumerate(class_dist.values):
    axes[1].text(class_dist.index[i], v + 1, str(v), ha='center')

plt.tight_layout()
plt.savefig('../reports/figures/mdr_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# MDR by organism
mdr_by_organism = mdr_df.groupby('Organism')['MDR_Status'].apply(
    lambda x: (x == 'MDR').sum() / len(x) * 100
).sort_values(ascending=False)

# Filter organisms with at least 5 isolates
org_counts = mdr_df['Organism'].value_counts()
mdr_by_organism_filtered = mdr_by_organism[mdr_by_organism.index.isin(org_counts[org_counts >= 5].index)]

print("\nMDR Rate by Organism (minimum 5 isolates):")
for org, rate in mdr_by_organism_filtered.items():
    n = org_counts[org]
    print(f"{org}: {rate:.1f}% (n={n})")

# Visualize
plt.figure(figsize=(10, 6))
bars = plt.barh(range(len(mdr_by_organism_filtered)), mdr_by_organism_filtered.values, 
                color='coral', alpha=0.7)
plt.yticks(range(len(mdr_by_organism_filtered)), mdr_by_organism_filtered.index)
plt.xlabel('MDR Rate (%)')
plt.title('MDR Rate by Organism', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

for i, (org, rate) in enumerate(mdr_by_organism_filtered.items()):
    n = org_counts[org]
    plt.text(rate + 2, i, f'{rate:.1f}% (n={n})', va='center')

plt.tight_layout()
plt.savefig('../reports/figures/mdr_by_organism.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Summary Statistics and Recommendations

In [ ]:
# Generate summary report
print("=" * 70)
print("ANTIMICROBIAL RESISTANCE PREVALENCE SUMMARY")
print("=" * 70)

print("\n1. OVERALL RESISTANCE PATTERNS")
print("-" * 70)
high_res = resistance_df[resistance_df['Resistance_Rate'] >= 50]
print(f"   Antibiotics with HIGH resistance (≥50%): {len(high_res)}")
if len(high_res) > 0:
    print(f"   Top 3: {', '.join(high_res['Antibiotic'].head(3).tolist())}")

low_res = resistance_df[resistance_df['Resistance_Rate'] < 20]
print(f"\n   Antibiotics with LOW resistance (<20%): {len(low_res)}")
if len(low_res) > 0:
    print(f"   Examples: {', '.join(low_res['Antibiotic'].head(5).tolist())}")

print("\n2. MULTI-DRUG RESISTANCE")
print("-" * 70)
print(f"   MDR Prevalence: {mdr_prevalence:.1f}%")
print(f"   MDR Isolates: {mdr_count}/{len(mdr_df)}")
print(f"   Interpretation: {'CRITICAL - Immediate stewardship needed' if mdr_prevalence > 40 else 'CONCERNING - Enhanced monitoring required' if mdr_prevalence > 20 else 'MODERATE'}")

print("\n3. TREATMENT IMPLICATIONS")
print("-" * 70)
first_line = resistance_df[resistance_df['Sensitivity_Rate'] >= 80]
print(f"   Potential first-line agents (≥80% sensitivity): {len(first_line)}")
if len(first_line) > 0:
    print(f"   Recommended: {', '.join(first_line['Antibiotic'].tolist())}")

avoid = resistance_df[resistance_df['Resistance_Rate'] >= 60]
print(f"\n   Antibiotics to avoid empirically (≥60% resistance): {len(avoid)}")
if len(avoid) > 0:
    print(f"   Avoid: {', '.join(avoid['Antibiotic'].tolist())}")

print("\n" + "=" * 70)

In [ ]:
# Export resistance summary
resistance_df.to_csv('../data/processed/resistance_rates_summary.csv', index=False)
mdr_df.to_csv('../data/processed/mdr_analysis.csv', index=False)
print("\nAnalysis results saved to data/processed/")